In [ ]:
# You must run this cell to install the Claude API
# You must have an account at claude.ai and you must be logged in through your browser
# and have allowed cookies on claude.ai to use this API.
!pip install unofficial-claude2-api

## Create prompts.json

In [ ]:
import json
# Open list of quotes from quotes.txt
with open("quotes.txt", 'r') as inp:
    data = inp.readlines()
    
# Open prompt template from prompt_template.txt
with open("prompt_template.txt", 'r') as templ:
    templ = templ.readlines()

In [ ]:
# Create prompt dictionary with definition, prompt, cue, and inserting 
cleaned_templ = [element for element in templ if element != "\n"]

dicts = []
counter = 1

for item in data:
    definition = cleaned_templ[0]
    prompt = cleaned_templ[1]
    cue = cleaned_templ[2]
    
    prompt_dict = {'prompt_num': counter, 'content': {'definition': definition, 'prompt': prompt, 'cue': cue, 'quote': item}}
    dicts.append(prompt_dict)
    counter += 1

In [ ]:
# Save prompt dictionary in json format
with open("prompts.json", 'w') as out:
    json.dump(dicts, out)

## Interfacing with Claude

In [ ]:
from claude2_api import *
import json
import time

# Open prompts.json
with open("prompts.json", 'r') as data:
    prompts = json.load(data)

In [ ]:
# You must have an account at claude.ai and you must be logged in through your browser
# and have allowed cookies on claude.ai before running this code.
session = get_session_data()
client = ClaudeAPIClient(session, timeout=240)

In [ ]:
responses = []

# Change whether to include or exclude definitions in prompts
include_defs = False

for item in prompts:
    chat_id = client.create_chat()
    
    message_to_send = (f"{item['content']['definition']}\n{item['content']['prompt']}\n{item['content']['cue']}\n{item['content']['quote']}") if include_defs == True else (f"{item['content']['prompt']}\n{item['content']['cue']}\n{item['content']['quote']}")
    res = client.send_message(chat_id, message_to_send)
    item['content']['response'] = res.answer
    if include_defs == False:
        del item['content']['definition']
    responses.append(item)
    print('Sleeping to avoid detection ;)')
    time.sleep(30)
    
if include_defs == False:
    with open("responses_no_def.json", 'w') as out:
        json.dump(responses, out)
else:
    with open("responses_with_def.json", 'w') as out:
        json.dump(responses, out)